In [61]:
import numpy as np
import time
from selenium import webdriver
from typing import List
import pandas as pd
import os
import datetime

In [62]:
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import urllib.request

chrome_options = Options()

In [63]:
BASE_URL = "https://dgsaie.mise.gov.it/open-data"
chromedriver = "/home/alesssandro/sw/chromedriver"
driver = webdriver.Chrome (executable_path="/home/alesssandro/sw/chromedriver")
browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chrome_options)
browser.get(BASE_URL)
wait = WebDriverWait(browser, 10)

<ipython-input-63-31e0dff60d03>:4: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path=chromedriver, chrome_options=chrome_options)


In [64]:
fuel_dictionary = {'DATA_RILEVAZIONE': [], 'BENZINA': [], 'GASOLIO_AUTO': [],'GPL': []}
date_path = ("//td[@class='google-visualization-table-type-date google-visualization-table-td']")
petrol_path = ("//td[@class='google-visualization-table-type-number google-visualization-table-td'][1]")
diesel_path = ("//td[@class='google-visualization-table-type-number google-visualization-table-td'][2]")
gpl_path = ("//td[@class='google-visualization-table-type-number google-visualization-table-td'][3]")



In [77]:
def getFile():
    url = 'https://dgsaie.mise.gov.it/open_data_export.php?export-id=1&export-type=csv'
    urllib.request.urlretrieve(url, '/home/alesssandro/Scrivania/fuelprice.csv')
    f=pd.read_csv("/home/alesssandro/Scrivania/fuelprice.csv")
    keep_col = ['DATA_RILEVAZIONE','BENZINA','GASOLIO_AUTO','GPL']#removing unnecessary columns
    new_f = f[keep_col]
    new_f.to_csv("/home/alesssandro/Scrivania/fuelprice.csv", index=False)

In [66]:
def get_values(list_name, path_name, dictionary_key):
    for value in browser.find_elements_by_xpath(path_name):
        list_name.append(value.text)  
   
    for line in list_name:
        fuel_dictionary["{}".format(dictionary_key)].append(line)
    

In [67]:
def monthToNum(month):
    return {
            'Gen': "01",'Feb': "02", 'Mar': "03", 'Apr': "04",
            'Mag': "05", 'Giu': "06",'Lug': "07", 'Ago': "08",
            'Set': "09", 'Ott': "10",'Nov': "11", 'Dic': "12"
    }[month]

In [68]:
def changeDateValue():
    cleaned_list = [] #temporary list with cleaned values
    values = fuel_dictionary["DATA_RILEVAZIONE"]
    
    for line in values:
        
        day = line.split(" ",2)[0]
        month = line.split(" ",2)[1]
        year = line.split(" ",2)[2]

        date_string = year + "-" + monthToNum(month) + "-" + day
        cleaned_list.append(date_string)
    fuel_dictionary["DATA_RILEVAZIONE"].clear()
    
    for date in cleaned_list:
        fuel_dictionary["DATA_RILEVAZIONE"].append(date)

In [69]:
def get_data():
    date_list = []
    petrol_list = []
    diesel_list = []
    gpl_list = []
    get_values(date_list, date_path, "DATA_RILEVAZIONE")
    get_values(petrol_list, petrol_path, "BENZINA")
    get_values(diesel_list, diesel_path, "GASOLIO_AUTO")
    get_values(gpl_list, gpl_path, "GPL")

In [70]:
def getLastRow():
    df1 = pd.read_csv('/home/alesssandro/Scrivania/fuelprice.csv')
    df2 = pd.DataFrame(df1.iloc[-1:,:].values) #get last row
    last_row = df2._get_value(0,0)#.replace("-", "") # date of last row
    return last_row
    

In [71]:
def getLastMonth():
    today = datetime.date.today()
    lastMonth = today - datetime.timedelta(days=31)
    lastMonth = lastMonth.strftime("%d%m%Y")
    return lastMonth

In [72]:
def noDuplicate(): #old method with open
    with open('/home/alesssandro/Scrivania/fuelprice.csv', 'r') as fp:
        s = fp.read()
    missing = []
    for url in fuel_dictionary:
        if url not in s:
            missing.append(url + '\n')
    if missing:
        with open('/home/alesssandro/Scrivania/fuelprice.csv', 'a+') as fp:
            fp.writelines(missing)

In [73]:
def removeDuplicate():
    getLastRow()
    for cicle in range(len(fuel_dictionary)-1):
        if last_row == fuel_dictionary['DATA_RILEVAZIONE'][cicle]:
            for key in fuel_dictionary.keys():
                    del fuel_dictionary[key][cicle]

In [75]:
def intoCsv():
    df2 = pd.DataFrame(data = fuel_dictionary)
    df2.to_csv('/home/alesssandro/Scrivania/fuelprice.csv', mode='a', index = None, header = False, decimal='.', sep=',')

In [74]:
try:
    file = open("/home/alesssandro/Scrivania/fuelprice.csv")
    browser.get("https://dgsaie.mise.gov.it/analisi-prezzi-settimanali-carburanti")
    #getLastRow()
    wait.until(EC.visibility_of_element_located((By.NAME, "DGISSEG_MiSE_wm_form_dati_prezzo_intervallo_da"))).send_keys(getLastMonth())
    wait.until(EC.visibility_of_element_located((By.ID, "DGISSEG_MiSE_wm_formcheck_1006"))).click();#GPL checkbox
    
    wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='mt-4 text-center']/button[@class='btn btn-block btn-primary wmSubmitButton text-break']"))).click(); 

    time.sleep(3)
    get_data()
    changeDateValue()
    removeDuplicate()
    intoCsv()
    
except IOError:
    getFile()